> combinations is 2**n, not n! think about why this is the case [...]

Let's redo this problem: https://leetcode.com/problems/combinations/.

In [10]:
def combine(n, k):
    return _combine(k, list(range(1, n + 1)))

def _combine(k, nums):
    if len(nums) == 0: return []
    if k == 0: return []
    if k > len(nums): return []

    if k == 1: return [[n] for n in nums]
    
    out = []
    for i in range(len(nums)):
        remaining_nums = nums[i + 1:]
        lead_num = nums[i]
        subanswers = _combine(k - 1, remaining_nums)
        result = [[lead_num] + s for s in subanswers]
        out += result
    return out

* This time I knew the combinations property and did this right on the first try and at speed.
* Ok says that my analysis of the runtime is wrong, and provides the following template for the purposes of evaluation.

In [21]:
def combinations(arr, i=0, cur=[], output=[]):
    if i >= len(arr):
        output.append(cur[:])
        return

    # add the current element
    cur.append(arr[i])
    combinations(arr, i + 1, cur, output)
    cur.pop()

    # left branch: skip the current element
    combinations(arr, i + 1, cur, output)

* I tried reanalyzing my version of this function but arrived at the same conclusion. I tried analyzing Ok's algorithm (I did not understand the logic immediately) and found that this template nets you all combinations of the numbers of any length and that it does indeed branch $2^n$ time, with an overall runtime he suggests.
* Despite a lot of time spent starting at a paper analysis of this problem, I don't see a way of arriving at this boundary using the code as I wrote it (this is a Q for Ok).
* When looking at the problem from a pick-any-k perspective, it's obvious. Each number can be picked or not picked. That's 2 options per number, and there are n numbers, and hence we get $2^n$.


In [30]:
def combinations(arr):
    if len(arr) == 0: return []
    if len(arr) == 1: return [[v] for v in arr]
    
    subcombinations = combinations(arr[1:])
    options_pick = [[arr[0]] + c for c in subcombinations]
    options_no_pick = subcombinations
    return options_pick + options_no_pick

In [31]:
combinations([1,2,3])

[[1, 2, 3], [1, 3], [2, 3], [3]]

* Here's my implementation of this idea, which is clearer to me than the former implementation, for whatever reason.
* Can we modify this to work for combinations only of a certain length? Obviously this will come down to restricting the base case against the `k` value.

In [40]:
def combinations(arr, k):
    if len(arr) == 0: return []
    if len(arr) < k: return []
    if k == 1: return [[v] for v in arr]
    
    options_pick = [[arr[0]] + c for c in combinations(arr[1:], k - 1)]
    options_no_pick = combinations(arr[1:], k)
    return options_pick + options_no_pick

In [45]:
combinations([1,2,3], 2)

[[1, 2], [1, 3], [2, 3]]

* I don't think I can see $O(n^2)$ in this answer either, except in knowing about the any-`k` case implementation, knowing this one is a subset of that implementation, and reasoning therefore that the lower bound for the any-`k` case must also be the lower bound for this code.

> for four sum: can you call into your two sum solution to solve it? what's the time complexity

First we reimplement `twoSum`.

In [47]:
def twoSum(nums, target):
    for i in range(len(nums)):
        for j in range(len(nums)):
            if i != j:
                if nums[i] + nums[j] == target:
                    return [i, j]

In [49]:
twoSum([2,3,5,7], 7)

[0, 2]

* This is $O(n^2)$.
* We can do this in $O(n\log{n})$ time with sorting, or in $O(n)$ time by using an additional memory store.

In [50]:
def twoSum(nums, target):
    outmap = {}
    for i in range(len(nums)):
        value_needed = target - nums[i]
        if value_needed in outmap:
            return [outmap[value_needed], i]
        else:
            outmap[nums[i]] = i

In [51]:
twoSum([2,3,5,7], 7)

[0, 2]

* Ok, so like, 4sum.

In [1]:
def fourSum(nums, target):
    outmap = {}
    for j in range(len(nums)):
        for k in range(j + 1, len(nums)):
            tot = nums[j] + nums[k]
            if tot not in outmap:
                outmap[tot] = [[j, k]]
            else:
                outmap[tot] = outmap[tot] + [[j, k]]
    
    out = []
    seen = set()
    for first_double_sum in outmap:
        for second_double_sum in outmap:
            for tup1 in outmap[first_double_sum]:
                for tup2 in outmap[second_double_sum]:
                    if len(set(tup1).intersection(set(tup2))) == 0:
                        sorted_quintuple = sorted(tup1 + tup2)
                        if first_double_sum + second_double_sum == target:
                            if tuple(sorted_quintuple) not in seen:
                                out.append(sorted_quintuple)
                                seen.add(tuple(sorted_quintuple))
    return [sorted([nums[i] for i in tup]) for tup in out]

* This solution is correct (absent some minor iteration order bug). But is it actually faster?
* First we double-iterate through the array to build an in-memory structure containing all pairs in the array by total: an $O(n^2)$ operation. Then we quadruple iterate over all combinations in the structure, comparing them to one another to try and find working quadruples. We know that visiting every combination of numbers in an array, as here, is an $O(2^n)$ operation; this implies that we are performing $O(n^2 + 2^n)$ work...but $2^n$ will eclipse $n^2$ at large values of $n$, so this solution is actually *worse* than the previous one.
* Actually if we take advantage of ordering the mapping we can do this in $O(n^2)$ work, but this solution is still too complex. Incomplete attempt below.

In [2]:
fourSum([1,2,3,4,100,125,1000], 10)

[[1, 2, 3, 4]]

In [3]:
fourSum([1, 0, -1, 0, -2, 2], 0)

[[-1, 0, 0, 1], [-2, -1, 1, 2], [-2, 0, 0, 2]]

In [4]:
list({1: [1,2]}.items())

[(1, [1, 2])]

In [5]:
def fourSum(nums, target):
    outmap = {}
    for j in range(len(nums)):
        for k in range(j + 1, len(nums)):
            tot = nums[j] + nums[k]
            if tot not in outmap:
                outmap[tot] = [[j, k]]
            else:
                outmap[tot] = outmap[tot] + [[j, k]]

    
    outmap_keys = sorted(list(outmap.keys()))
    j = 0
    k = len(outmap_keys) - 1
    
    out = []
    out_seen = set()
    while j < k:
        s = outmap_keys[j] + outmap_keys[k]
        if s == target:
            for a in outmap[outmap_keys[j]]:
                for b in outmap[outmap_keys[k]]:
                    quadruple = [nums[a[0]], nums[a[1]], nums[b[0]], nums[b[1]]]
                    if len(set(a + b)) == 4 and tuple(quadruple) not in out_seen:
                        out += [quadruple]
                        out_seen.add(tuple(quadruple))
            j += 1
            k -= 1
        elif s < target:
            j += 1
        else:  # s > target
            k -= 1
    
    return out

In [6]:
fourSum([1, 0, -1, 0, -2, 2], 0)

[[-1, -2, 1, 2], [0, -2, 0, 2], [1, -2, -1, 2], [0, -1, 1, 0], [-1, 0, 1, 0]]

In [7]:
def fourSum(nums, target):
    outmap = {}
    for j in range(len(nums)):
        for k in range(j + 1, len(nums)):
            tot = nums[j] + nums[k]
            if tot not in outmap:
                outmap[tot] = [[j, k]]
            else:
                outmap[tot] = outmap[tot] + [[j, k]]
    
    out = []
    for v in outmap:
        for pairA in outmap[v]:
            pairB = twoSum(
                nums[:pairA[0]] + nums[pairA[0] + 1:pairA[1]] + nums[pairA[1] + 1:],
                target - (pairA[0] + pairA[1])
            )
            if pairB is not None:
                out += [pairA + pairB]
    return out

In [9]:
# fourSum([1, 0, -1, 0, -2, 2], 0)

* The answer is something like this. Call `twoSum` as a subroutine to complete the existing pair. However I am tired of staring at this problem so I'm not fixing this implementation further.
* Here's the actual solution, courtesy of the comments:

In [10]:
def fourSum(self, nums, target):
    def findNsum(nums, target, N, result, results):
        if len(nums) < N or N < 2 or target < nums[0]*N or target > nums[-1]*N:  # early termination
            return
        if N == 2: # two pointers solve sorted 2-sum problem
            l,r = 0,len(nums)-1
            while l < r:
                s = nums[l] + nums[r]
                if s == target:
                    results.append(result + [nums[l], nums[r]])
                    l += 1
                    while l < r and nums[l] == nums[l-1]:
                        l += 1
                elif s < target:
                    l += 1
                else:
                    r -= 1
        else: # recursively reduce N
            for i in range(len(nums)-N+1):
                if i == 0 or (i > 0 and nums[i-1] != nums[i]):
                    findNsum(nums[i+1:], target-nums[i], N-1, result+[nums[i]], results)

    results = []
    findNsum(sorted(nums), target, 4, [], results)
    return results

> https://leetcode.com/problems/network-delay-time/ (djikstra)

* This question requires Djikstra's algorithm, so before we do any implementation for it we need to study Djikstra's algorithm in some depth.
* After spending some...OK, a lot of time just spinning the wheels with Djikstra, I realized a few things.
  * One, I really need to spend more time with data structures, and have them and their properties at hand, and that this need precludes working on Djikstra.
  * Two, I need to have a specific step in my solve workflow for defining the data structures I will use for the solve, and understanding how they flow through the solution. Data structures, not the algorithm itself, was the source of my inability to implement Djikstra right away.